# Section 1: Data Upload

In this section, you will upload your data to be used for analysis. Before uploading the data please ensure that you have installed Anaconda on your computer. If you have not already done so, follow the instructions here:
https://www.anaconda.com/products/individual

Alternatively, if you would like to proceed without installing Anaconda then please ensure the following packages are installed: tkinter, pandas, numpy, matlibplot.

Once Anaconda is installed, download the .ipynb file and open a jupyter notebook on your computer. Next, open the .ipynb in the jupyter notebook. At this point, you are ready to run the geochem script!

When you are ready to upload your data, click on the cell below and press the run button. The data must be one of the following data types: xls, xlsx, xlsm, xlsb, csv, and odf.

In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import math 
import seaborn as sns
import matplotlib.pyplot as plt


# Creating prompt for user to select file using tkinter package.
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

# Allowable excel file types for pandas dataframe.
file_types_excel = ['xls','xlsx','xlsm','xlsb','odf']

# Check if file is an excel file or an csv.
if file_path[-3:] in file_types_excel or file_path[-4:] in file_types_excel:
    df = pd.read_excel(file_path)
    print('Excel file uploaded successfully!')
 
elif file_path[-3:] == 'csv':
    df = pd.read_csv(file_path, low_memory = False)
    print('CSV file uploaded successfully!')
    
else:
    print("File type not supported. xls, xlsx, xlsm, xlsb, csv, and odf files are accepted.")

# Section 2: Numerical Transformations

In this section, the uploaded data will be transformed in the following ways:

Negative values --> Invert sign and divide value by 2. <br>
Zero values     --> Replace with a null value (NaN). <br>
Less than sign(e.g. <50) --> Remove < and divide value by 2. <br> 
Text values     --> Remove text value and replace with null (NaN). <br>


Run the module below to enter the units of the columns you would like to transform. <br> Be aware that the program is case sensitive. <br>
Example: Entering ppb will transform all columns with ppb in the title (au_ppb_ano, al_ppb_ano, b_ppb_ano would all be transformed.)  

Alternatively, each column could be marked with a character only used in those columns. In this case, just enter the identifying character. <br>
Example: Put || at the end of each column to be transformed (au_ppb_ano||, al_ppm_ano||, etc.) 
In this case, enter || in the module below. 

Lastly, type end when finished. 


In [ ]:
data_types = [] 
cur_in = ''

while True:
    cur_in = input('Enter column marker:')
    if cur_in == 'end':
        break
    else:
        data_types.append(cur_in)

print("Columns to be transformed:{}".format(data_types))

Run the module below to transform the selected columns.

In [ ]:
# Refresh df so that this module may be run multiple times.
#if file_path[-3:] in file_types_excel or file_path[-4:] in file_types_excel:
#    df = pd.read_excel(file_path, low_memory = False)
    
#elif file_path[-3:] == 'csv':
#    df = pd.read_csv(file_path, low_memory = False)

# Adding comment row to the dataframe, df, as row n+1. The format of the comments list is:
# [1.{# of instances}--Negative value, replaced with 1/2 Detection limit ---> NR
#  2.{# of instances}--Zero Changed to null   ---> ZR
#  3.{# of instances}--Less than (<) replaced with 1/2 Detection Limit ---> LTR
#  4.{# of instances}--Text replaced with null] ---> TR
# First, the number of instances will be collected, then the text will be added at the end. 
commentsdf = pd.DataFrame([[[0,0,0,0] for i in range(df.shape[1])]], columns = df.columns)
df = df.append(commentsdf, ignore_index = True)

# Function to check in current column is in the data types to be transformed
def data_bool(data_types_list, column):
    
    for i in range(len(data_types_list)):
        if data_types_list[i] in column:
            return True
    
    return False
 
print('Transforming... ')

neg_num = [-9,-99,-999,-9999,-99999]

#Search all columns. If column, col, is in data_types then transform the column.
for col in df.columns:
    if data_bool(data_types, col):
        for i in range(df.shape[0]-1):
            
            
            if str(df.at[i, col])[0] == '-' and str(df.at[i, col])[-1] == '9' or df.at[i, col] in neg_num:
    
                df.at[i, col] = np.NaN
            
            #check if data point is zero. Increment the ZR counter one. 
            if df.at[i, col] == 0:
                df.at[i, col] = np.NaN
                df.at[df.shape[0]-1, col][1] += 1

            #check if data point begins with the less than symbol. Must use try as int's are unscriptable. 
            #increment the LTR counter one. 
            try:
                if df.at[i,col][0] == '<':
                    df.at[i,col] = int(df.at[i,col].replace('<',''))/2
                    df.at[df.shape[0]-1, col][2] += 1
            except:
                pass
            
            #check if data point is negative. Use try because a string can't be compared with <. 
            #increment the NR counter.
            try:
                if df.at[i,col] < 0:
                    df.at[i,col] = df.at[i,col]*(-0.5)
                    df.at[df.shape[0]-1, col][0] += 1
            except:
                pass

            #check if data point has text in it. If it does, then calling float will give an error. 
            #increment the TR counter one.
            try: 
                float(df.at[i,col])

            except:
                df.at[i,col] = np.NaN
                df.at[df.shape[0]-1, col][3] += 1
    else: 
        pass


#format the comments section with descriptions.

for col in df.columns:
 
    df.at[df.shape[0]-1, col][0] = "NR = {}".format(df.at[df.shape[0]-1, col][0])
    df.at[df.shape[0]-1, col][1] = "ZR = {}".format(df.at[df.shape[0]-1, col][1])
    df.at[df.shape[0]-1, col][2] = "LTR = {}".format(df.at[df.shape[0]-1, col][2])
    df.at[df.shape[0]-1, col][3] = "TR = {}".format(df.at[df.shape[0]-1, col][3])

print('Numerical transformations complete!')


# Section 3: Data Visualization 

Run the module below to select the folder for storing the figures.

In [ ]:
root = tk.Tk()
root.withdraw()
folder_selected = filedialog.askdirectory()

print("Folder Selected: " + folder_selected)

In this section, there are three different inputs:

1. Enter the column that you would like the data plotted against.
2. Enter the elements to be plotted on a linear scale (enter end when finished.)
3. Enter the elements to be plotted on a log scale (enter end when finished.)

Please ensure that the rows begin with the element and are followed by an underscore. <br>
Example: cu_ppm_ano

In [ ]:
log_elements = [] 
lin_elements = []
cur_in = ''

rock_type = input('Enter row to be plotted against data: ')

print('Enter elements to be plotted on a linear scale.')
while True:
    cur_in = input('Enter element:')
    if cur_in == 'end':
        break
    else:
        lin_elements.append(cur_in + '_')

print('Enter elements to be plotted on a log scale.')
while True:
    cur_in = input('Enter element:')
    if cur_in == 'end':
        break
    else:
        log_elements.append(cur_in + '_')

print("Elements to be graphed on a linear scale:{}".format(lin_elements))
print("Elements to be graphed on a log scale:{}".format(log_elements))

The figures are ready to be generated! Run the module below and the figures will be saved to your computer.

In [ ]:
#Taking only the first rock type from the aligned_ro row to be analyzed.
import matplotlib.style as style 
sns.set(context = 'paper') 


#parsing the  column so that only the first rock type is used. E.G. for (basalt, dacite) only basalt would be used. 
for i in range(df.shape[0]-1):   
    if ',' in df.at[i, rock_type]:
        comma_index = df.at[i, rock_type].find(',')
        df.at[i, rock_type] = df.at[i, rock_type][:comma_index-1]

# Making a copy of the dataframe that doesn't contain the comments row. This is done so the columns may be graphed 
# without raising datatype errors. 
df_no_comments = df.copy()
df_no_comments = df_no_comments.drop(df.shape[0]-1)

# finding all columms in the dataframe which begin with the elements to be graphed. "_" is added so that only the 
# element columns are found. 

"""log_elements = ['ag_','as_','au_','be_','bi_','cd_','co_','cr_','cu_','ge_','hf_','hg_','in_','la_','mo_','ni_',\
         'pb_','pd_','pt_','rb_','re_','sb_','se_','sr_','ta_','te_','th_','ti_','u_','v_','w_','y_','zn_','zr_']
lin_elements = ['al_','b_','ba_','ca_','ce_','cs_','fe_','ga_','k_','li_',"mg_",'mn_','na_','nb_','p_','rb_',\
                's_','sc_','sn_','ti_']"""

log_col = []  #contains all columns to be plotted with a log scale.
lin_col = []  #contains all columns to be plotted with a lin scale. 

for col in df_no_comments.columns:
    if col[:2] in log_elements or col[:3] in log_elements:
        log_col.append(col)
for col in df_no_comments.columns:
    if col[:2] in lin_elements or col[:3] in lin_elements:
        lin_col.append(col)


rocks = [] # Contains one of each rock type. This is done so that the rock types can be shown on the plots in alphabetical order. 
for x in df_no_comments['aligned_ro']:
    if x not in rocks:
        rocks.append(x)

print('Making linear figures...')

for col in df_no_comments.columns:
    if col in lin_col:
        
    
        plt.figure(figsize=(200,200))
        plt.title("{} - By Grouped Rock Type".format(col), fontsize =300)
        a = sns.boxplot(pd.to_numeric(df_no_comments[col]), df_no_comments[rock_type], order = sorted(rocks), palette = 'muted')
        a.tick_params(labelsize=200)
        a.set_xlabel(col ,fontsize= 300)
        a.set_ylabel(rock_type , fontsize=300)
        plt.tight_layout()
        path = folder_selected + "/" + col
        a.get_figure().savefig(path)
        plt.close()


print('Making log figures...')

for col in df_no_comments.columns:
    if col in log_col:
       
        plt.figure(figsize=(150,200))
        plt.title("{} - By Grouped Rock Type".format(col), fontsize =300)
        a = sns.boxplot(pd.to_numeric(df_no_comments[col]), df_no_comments[rock_type], order = sorted(rocks), palette = 'muted')
        a.tick_params(labelsize=200)
        a.set_xlabel(col ,fontsize= 300)
        a.set_ylabel(rock_type , fontsize=300)
        path = folder_selected + "/" + col
        a.get_figure().savefig(path)
        plt.close()


print('Done ')